# pangeo ERA data

In [ ]:
#notebook based on zflemings:https://nbviewer.jupyter.org/github/zflamig/dask-era5/blob/main/notebook/era5_fargate_dask.ipynb
import xarray as xr
import fsspec
import dask
import s3fs
import numpy as np
xr.set_options(display_style="html")  #display dataset nicely 

In [ ]:
import gcsfs
import fsspec
import xarray as xr

with xr.open_zarr(fsspec.get_mapper('gcs://pangeo-era5/reanalysis/spatial-analysis'),
                  consolidated=True,
                  chunks='auto') as ds:
    print(ds)

In [ ]:

from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/atmosphere.yaml")
ds  = cat["era5_hourly_reanalysis_single_levels_sa"].to_dask()
ds_wnd = ds.u10

In [ ]:
ds_wnd

In [ ]:
ds_wnd[0,:,:].plot()

In [ ]:
ts = ds_wnd.sel(longitude=slice(236,239),latitude=slice(46,41)).mean({'latitude','longitude'}).load()

In [ ]:
ts

In [ ]:
ts.to_netcdf('../../data/era5ts.nc')

In [ ]:
sub=ts.sel(time=slice('2017','2020'))

# AWS ERA5 data

In [ ]:
# see if zac's data more recent?
#https://nbviewer.jupyter.org/github/awslabs/amazon-asdi/blob/main/examples/dask/notebooks/era5_zarr.ipynb

%matplotlib inline
import boto3
import botocore
import datetime
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
import numpy as np
import s3fs
import fsspec
import dask
from dask.distributed import performance_report, Client, progress

font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 18}
matplotlib.rc('font', **font)

In [ ]:
#zac code
def fix_accum_var_dims(ds, var):
    # Some varibles like precip have extra time bounds varibles, we drop them here to allow merging with other variables
    
    # Select variable of interest (drops dims that are not linked to current variable)
    ds = ds[[var]]  

    if var in ['air_temperature_at_2_metres',
               'dew_point_temperature_at_2_metres',
               'air_pressure_at_mean_sea_level',
               'northward_wind_at_10_metres',
               'eastward_wind_at_10_metres']:
        
        ds = ds.rename({'time0':'valid_time_end_utc'})
        
    elif var in ['precipitation_amount_1hour_Accumulation',
                 'integral_wrt_time_of_surface_direct_downwelling_shortwave_flux_in_air_1hour_Accumulation']:
        
        ds = ds.rename({'time1':'valid_time_end_utc'})
        
    else:
        print("Warning, Haven't seen {var} varible yet! Time renaming might not work.".format(var=var))
        
    return ds

@dask.delayed
def s3open(path):
    fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False, 
                           config_kwargs = {'max_pool_connections': 20})
    return s3fs.S3Map(path, s3=fs)


def open_era5_range(start_year, end_year, variables):
    ''' Opens ERA5 monthly Zarr files in S3, given a start and end year (all months loaded) and a list of variables'''
    
    
    file_pattern = 'era5-pds/zarr/{year}/{month}/data/{var}.zarr/'
    
    years = list(np.arange(start_year, end_year+1, 1))
    months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
    
    l = []
    for var in variables:
        print(var)
        
        # Get files
        files_mapper = [s3open(file_pattern.format(year=year, month=month, var=var)) for year in years for month in months]
        
        # Look up correct time dimension by variable name
        if var in ['precipitation_amount_1hour_Accumulation']:
            concat_dim='time1'
        else:
            concat_dim='time0'
            
        # Lazy load
        ds = xr.open_mfdataset(files_mapper, engine='zarr', 
                               concat_dim=concat_dim, combine='nested', 
                               coords='minimal', compat='override', parallel=True)
        
        # Fix dimension names
        ds = fix_accum_var_dims(ds, var)
        l.append(ds)
        
    ds_out = xr.merge(l)
    
    return ds_out



In [ ]:
%%time

ds = open_era5_range(1979, 2020, ["eastward_wind_at_10_metres"])

In [ ]:
ds

In [ ]:
ds.eastward_wind_at_10_metres.sel(lon=slice(235,245),lat=slice(45,35),valid_time_end_utc='2017-10-08T18').plot(vmin=-3,vmax=5,cmap='RdBu_r')

In [ ]:
ts = ds.sel(lon=slice(236,239),lat=slice(46,41)).mean({'lat','lon'}).load()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
fig, ax = plt.subplots(figsize=(12, 6))
for lyr in range(1979,2021):
    yr = ts.sel(valid_time_end_utc=slice(str(lyr)+'-08-01',str(lyr)+'-10-29'))
    xx=yr.valid_time_end_utc.dt.dayofyear+yr.valid_time_end_utc.dt.hour/24
    ax.plot(xx,yr.eastward_wind_at_10_metres,color='silver')
color_order = ['green','grey','blue','red']
for i,lyr in enumerate(np.arange(2017,2021)):
    yr = ts.sel(valid_time_end_utc=slice(str(lyr)+'-08-01',str(lyr)+'-10-29'))
    ax.plot(xx,yr.eastward_wind_at_10_metres,color=color_order[i])
date_form = DateFormatter("%m-%d")
ax.xaxis.set_major_formatter(date_form)

In [ ]:
ts_sr = ds.sel(lon=slice(237,240),lat=slice(42,40)).mean({'lat','lon'}).load()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
fig, ax = plt.subplots(figsize=(12, 6))
for lyr in range(1979,2021):
    yr = ts_sr.sel(valid_time_end_utc=slice(str(lyr)+'-08-01',str(lyr)+'-10-29'))
    xx=yr.valid_time_end_utc.dt.dayofyear+yr.valid_time_end_utc.dt.hour/24
    ax.plot(xx,yr.eastward_wind_at_10_metres,color='silver')
color_order = ['green','grey','blue','red']
for i,lyr in enumerate(np.arange(2017,2021)):
    yr = ts_sr.sel(valid_time_end_utc=slice(str(lyr)+'-08-01',str(lyr)+'-10-29'))
    ax.plot(xx,yr.eastward_wind_at_10_metres,color=color_order[i])
date_form = DateFormatter("%m-%d")
ax.xaxis.set_major_formatter(date_form)

In [ ]:
ts.to_netcdf('../../data/era5ts.nc')
ts_sr.to_netcdf('../../data/era5ts_sr.nc')